# Batch evaluation with your own data
The following sample shows the basic way to evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration,
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator,
)

from azure.ai.ml import MLClient

load_dotenv(override=True)

## 🚀 Run Evaluators for local and upload to cloud (azure.ai.evaluation.evaluate)

In [ ]:
credential = DefaultAzureCredential()

azure_ai_project_conn_str = os.environ.get("AZURE_AI_PROJECT_CONN_STR")
subscription_id = azure_ai_project_conn_str.split(";")[1]
resource_group_name = azure_ai_project_conn_str.split(";")[2]
project_name = azure_ai_project_conn_str.split(";")[3]

azure_ai_project_dict = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=azure_ai_project_conn_str
)

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

## 🚀 Generate response dataset with Azure OpenAI
- Use your models to generate answers based on the query data set. These response records serve as the seed for creating assessments. By customizing your prompts, you can produce text tailored to your domain.

In [ ]:
from openai import AzureOpenAI

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

try:
    client = AzureOpenAI(
        azure_endpoint=aoai_api_endpoint,
        api_key=aoai_api_key,
        api_version=aoai_api_version,
    )

    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")

except (ValueError, TypeError) as e:
    print(e)

## Case1. When you have all the data (Query + Response + Ground Truth)
- If you have all the data, you can evaluate the model / service with the following steps.
- Assume that you already upload the data file (csv) to Azure Blob storage 

In [ ]:
from azure.storage.blob import BlobServiceClient


def upload_evaldata_to_blob(save_path, blob_name, container_name="eval-container"):
    # Create a blob client using the local file name as the name for the blob
    # Retrieve the storage connection string from environment variables
    blob_conn_str = os.getenv("AZURE_STORAGE_BLOB_CONNECTION_STRING")
    if not blob_conn_str or blob_conn_str == "":
        raise ValueError("AZURE_STORAGE_BLOB_CONNECTION_STRING must be set.")

    blob_service_client = BlobServiceClient.from_connection_string(blob_conn_str)

    if not blob_service_client.get_container_client(container_name).exists():
        blob_service_client.create_container(container_name)

    container_client = blob_service_client.get_container_client(container_name)

    # Upload the created file
    with open(f"data/{blob_name}", "rb") as data:
        container_client.upload_blob(blob_name, data, overwrite=True)
    blob_client = container_client.get_blob_client(blob_name)

    # Download CSV from Azure Blob and save locally
    with open(save_path, "wb") as f:
        blob_data = blob_client.download_blob()
        blob_data.readinto(f)
    print(f"Downloaded blob data to {save_path}")

In [ ]:
blob_name = "eval_all_data.csv"
container_name = "eval-container"
save_path = "case1_temp_data.csv"
upload_evaldata_to_blob(save_path, blob_name, container_name)
df = pd.read_csv(save_path)
df.head(5)

In [ ]:
# Create a jsonl file from the query data
outname = "case1_all_data.jsonl"

outdir = "./data"
if not os.path.exists(outdir):
    os.mkdir(outdir)

input_path = os.path.join(outdir, outname)
df.to_json(input_path, orient="records", lines=True, force_ascii=False)

## Case2. When you have only query and ground truth data (Query + Ground Truth)
- If you have all the data, you can generate the response and evaluate the model / service with the following steps.
- Assume that you already upload the data file (csv) to Azure Blob storage 

In [ ]:
blob_name = "eval_all_data.csv"
container_name = "eval-container"
save_path = "case2_temp_data.csv"
upload_evaldata_to_blob(save_path, blob_name, container_name)
df = pd.read_csv(save_path)
df.head(5)

In [ ]:
# Create a jsonl file from the query data
outname = "case2_temp_data.jsonl"

outdir = "./data"
if not os.path.exists(outdir):
    os.mkdir(outdir)

query_path = os.path.join(outdir, outname)
df.to_json(query_path, orient="records", lines=True, force_ascii=False)

In [ ]:
import tqdm

# This is the final jsonl file with the response added
# it will be used for evaluation
input_path = "./data/case2_query_response_data.jsonl"
with open(input_path, "w", encoding="utf-8") as outfile:
    outfile.write("")


with open(query_path, "r", encoding="utf-8") as infile, open(
    input_path, "a", encoding="utf-8"
) as outfile:

    for idx, line in enumerate(infile):
        print(f"=== Processing line {idx} ===")
        data = json.loads(line)
        resp = client.chat.completions.create(
            model=aoai_deployment_name,
            messages=[{"role": "user", "content": data["query"]}],
            temperature=0,
        )

        response_text = resp.choices[0].message.content
        print(response_text)
        data["response"] = response_text

        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")

## Evaluate the model / service with the set up data 
- Check that you have already run case1 or case2 as input data.

In [ ]:
input_path = input_path  # be sure which case you took
output_path = "./data/cloud_evaluation_output.json"


# https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/flow-evaluate-sdk
retrieval_evaluator = RetrievalEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

column_mapping = {
    "query": "${data.query}",
    "ground_truth": "${data.ground_truth}",
    "response": "${data.response}",
    "context": "${data.context}",
}

In [ ]:
import datetime

result = evaluate(
    evaluation_name=f"evaluation_local_upload_cloud_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    data=input_path,
    evaluators={
        "groundedness": groundedness_evaluator,
        "retrieval": retrieval_evaluator,
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator,
        "fluency": fluency_evaluator,
        "similarity": similarity_evaluator,
    },
    evaluator_config={
        "groundedness": {"column_mapping": column_mapping},
        "retrieval": {"column_mapping": column_mapping},
        "relevance": {"column_mapping": column_mapping},
        "coherence": {"column_mapping": column_mapping},
        "fluency": {"column_mapping": column_mapping},
        "similarity": {"column_mapping": column_mapping},
    },
    azure_ai_project=azure_ai_project_dict,
    output_path=output_path,
)

In [ ]:
!az login --scope https://graph.microsoft.com//.default

## 🚀 Run Evaluators in Azure Cloud (azure.ai.projects.models.Evaluation)

In [ ]:
# id for each evaluator can be found in your AI Studio registry - please see documentation for more information
# init_params is the configuration for the model to use to perform the evaluation
# data_mapping is used to map the output columns of your query to the names required by the evaluator
# Evaluator parameter format - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluator-parameter-format
evaluators_cloud = {
    "f1_score": EvaluatorConfiguration(
        id=F1ScoreEvaluator.id,
    ),
    "relevance": EvaluatorConfiguration(
        id=RelevanceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={
            "query": "${data.query}",
            "context": "${data.context}",
            "response": "${data.response}",
        },
    ),
    "groundedness": EvaluatorConfiguration(
        id=GroundednessEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={
            "query": "${data.query}",
            "context": "${data.context}",
            "response": "${data.response}",
        },
    ),
    # "retrieval": EvaluatorConfiguration(
    #     #from azure.ai.evaluation._evaluators._common.math import list_mean_nan_safe\nModuleNotFoundError: No module named 'azure.ai.evaluation._evaluators._common.math'
    #     id=RetrievalEvaluator.id,
    #     #id="azureml://registries/azureml/models/Retrieval-Evaluator/versions/2",
    #     init_params={"model_config": model_config},
    #     data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    # ),
    "coherence": EvaluatorConfiguration(
        id=CoherenceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "response": "${data.response}"},
    ),
    "fluency": EvaluatorConfiguration(
        id=FluencyEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={
            "query": "${data.query}",
            "context": "${data.context}",
            "response": "${data.response}",
        },
    ),
    "similarity": EvaluatorConfiguration(
        # currently bug in the SDK, please use the id below
        # id=SimilarityEvaluator.id,
        id="azureml://registries/azureml/models/Similarity-Evaluator/versions/3",
        init_params={"model_config": model_config},
        data_mapping={
            "query": "${data.query}",
            "response": "${data.response}",
            "ground_truth": "${data.ground_truth}",
        },
    ),
}

### Data
- The following code demonstrates how to upload the data for evaluation to your Azure AI project. Below we use evaluate_test_data.jsonl which exemplifies LLM-generated data in the query-response format expected by the Azure AI Evaluation SDK. For your use case, you should upload data in the same format, which can be generated using the Simulator from Azure AI Evaluation SDK.

- Alternatively, if you already have an existing dataset for evaluation, you can use that by finding the link to your dataset in your registry or find the dataset ID.

In [ ]:
# # Upload data for evaluation
data_id, _ = azure_ai_project_client.upload_file("data/evaluate_test_data.jsonl")
# data_id = "azureml://registries/<registry>/data/<dataset>/versions/<version>"
# To use an existing dataset, replace the above line with the following line
# data_id = "<dataset_id>"

### Configure Evaluators to Run
- The code below demonstrates how to configure the evaluators you want to run. In this example, we use the F1ScoreEvaluator, RelevanceEvaluator and the ViolenceEvaluator, but all evaluators supported by Azure AI Evaluation are supported by cloud evaluation and can be configured here. You can either import the classes from the SDK and reference them with the .id property, or you can find the fully formed id of the evaluator in the AI Studio registry of evaluators, and use it here. 

In [ ]:
evaluation = Evaluation(
    display_name=f"evaluation_cloud_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    description="Cloud Evaluation of dataset",
    data=Dataset(id=data_id),
    evaluators=evaluators_cloud,
)

# Create evaluation
evaluation_response = azure_ai_project_client.evaluations.create(
    evaluation=evaluation,
)

In [ ]:
from tqdm import notebook
import time


# Monitor the status of the run_result
def monitor_status(project_client: AIProjectClient, evaluation_response_id: str):
    with notebook.tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = project_client.evaluations.get(evaluation_response_id).status
        if status == "Queued":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            notebook.tqdm.write(f"Current Status: {status}")
            time.sleep(10)
            status = project_client.evaluations.get(evaluation_response_id).status
        while pbar.n < 3:
            pbar.update(1)
        notebook.tqdm.write("Operation Completed")

In [ ]:
monitor_status(azure_ai_project_client, evaluation_response.id)

### Check the evaluation result in Azure AI Foundry 
- After running the evaluation, you can check the evaluation results in Azure AI Foundry. You can find the evaluation results in the Evaluation tab of your project.

In [ ]:
# Get evaluation
get_evaluation_response = azure_ai_project_client.evaluations.get(
    evaluation_response.id
)

print("----------------------------------------------------------------")
print("Created evaluation, evaluation ID: ", get_evaluation_response.id)
print("Evaluation status: ", get_evaluation_response.status)
print(
    "AI Foundry Portal URI: ",
    get_evaluation_response.properties["AiStudioEvaluationUri"],
)
print("----------------------------------------------------------------")

![Cloud Evaluation Result](./images/cloud_evaluation_result.png)

In [ ]:
print("----------------------------------------------------------------")
print("Created evaluation, evaluation ID: ", get_evaluation_response.id)
print("Evaluation status: ", get_evaluation_response.status)
print(
    "AI Foundry Portal URI: ",
    get_evaluation_response.properties["AiStudioEvaluationUri"],
)
print("-----------------------------------")